In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import PyPDF2
import pandas as pd
import traceback
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key = openai_api_key,model_name = "gpt-3.5-turbo",temperature = 0.7)

C:\Users\Dell\AppData\Local\Temp\ipykernel_11896\3429325067.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key = openai_api_key,model_name = "gpt-3.5-turbo",temperature = 0.7)


In [4]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [5]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}"""

In [6]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [7]:
quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_11896\3646152006.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt, output_key="quiz", verbose=True)


In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [9]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [ ]:
review_chain = LLMChain(llm=llm, prompt = quiz_evaluation_prompt, output_key="review", verbose = True)

In [11]:
main_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"], output_variables=["quiz","review"],verbose=True)

In [12]:
PATH = "D:\MCQ-Generator\MCQ-Generator\data.txt"

with open(PATH,"r")as file:
    TEXT = file.read()
    

In [13]:
NUMBER = 5
SUBJECT = "Generative AI"
TONE = "simple"
RESPONSE_JSON = RESPONSE_JSON

In [14]:
with get_openai_callback() as cb:
    response=main_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\Dell\AppData\Local\Temp\ipykernel_11896\2399507670.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=main_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora.[9][10][11][12] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous

In [15]:
print(cb)

Tokens Used: 1666
	Prompt Tokens: 1162
		Prompt Tokens Cached: 0
	Completion Tokens: 504
		Reasoning Tokens: 0
Successful Requests: 2
Total Cost (USD): $0.002751


In [16]:
response

{'text': 'Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]\n\nImprovements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the 2020s. These include chatbots such as ChatGPT, Copilot, Gemini, and LLaMA; text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video AI generators such as Sora.[9][10][11][12] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[7][13][14]\n\nGenerative AI has uses across 

In [17]:
quiz = json.loads(response.get('quiz'))

In [18]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [19]:
quiz_table_data

[{'MCQ': 'What is Generative AI?',
  'Choices': 'a: A subset of artificial intelligence that uses generative models to produce data | b: A subset of artificial intelligence that focuses on robotics | c: A subset of artificial intelligence that analyzes existing data | d: A subset of artificial intelligence that focuses on speech recognition',
  'Correct': 'a'},
 {'MCQ': 'What enabled an AI boom of generative AI systems in the 2020s?',
  'Choices': 'a: Improvements in transformer-based deep neural networks | b: Decrease in AI research funding | c: Lack of interest in AI technologies | d: Focus on traditional programming methods',
  'Correct': 'a'},
 {'MCQ': 'Which company is NOT mentioned as a developer of generative AI models in the text?',
  'Choices': 'a: OpenAI | b: Microsoft | c: Apple | d: Google',
  'Correct': 'c'},
 {'MCQ': 'What are some of the industries where Generative AI has uses?',
  'Choices': 'a: Healthcare and finance only | b: Entertainment and customer service only | 

In [22]:
(pd.DataFrame(quiz_table_data)).to_csv("quiz.csv", index=False)